<a href="https://colab.research.google.com/github/bdeekshith066/blind/blob/main/blind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pyngrok

In [ ]:
!apt-get update -y
!apt-get install -y poppler-utils python3 python3-venv tesseract-ocr tesseract-ocr-kan
!pip install --upgrade AksharaJaana
!pip install pytesseract
!pip install easyocr
!pip install paddleocr
!pip install paddlepaddle
!pip install ultralytics

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 233 kB in 2s (104 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-k

In [ ]:
!pip install gtts

In [30]:
import getpass
import os
import threading

from flask import Flask, render_template, request, send_file, url_for, send_from_directory
from pyngrok import ngrok, conf
from werkzeug.utils import secure_filename
import base64
from PIL import Image
from ultralytics import YOLO
import os
from AksharaJaana.main import OCREngine
from AksharaJaana.utils import ModelTypes, FileOperationUtils
from gtts import gTTS

output_dir = 'kannada_output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
mkdir templates

mkdir: cannot create directory ‘templates’: File exists


In [ ]:
mkdir static

mkdir: cannot create directory ‘static’: File exists


In [ ]:
%ls

best.pt  index.html  kannada_output/  sample_data/  static/  templates/


In [ ]:
# Load the YOLOv8 model
model = YOLO('/content/best.pt')

In [39]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

UPLOAD_FOLDER = 'static/uploads'  # Specify the directory you want to save the images
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Define Flask routes
@app.route("/", methods=['GET', 'POST'])
def index():
    image_path = None
    if request.method == 'POST':
      if 'image' not in request.files:
        return 'No file part'
      image_file = request.files['image']
      if image_file.filename == '':
        return 'No selected file'
      if image_file:
        filename = 'input.jpg'
        image_file.save(os.path.join(os.getcwd(), app.config['UPLOAD_FOLDER'], filename))
        image_path = url_for('static', filename=os.path.join('uploads', filename))
      # Run inference on the image
      results = model('/content/static/uploads/input.jpg', conf = 0.6, classes = [0])
      original_image = Image.open('/content/static/uploads/input.jpg')
      with open(os.path.join(app.config['UPLOAD_FOLDER'], 'input.jpg'), "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')
      image_path1 = '/content/static/uploads/input.jpg'
      with open(image_path1, "rb") as image_file:
        clipart_base64 = base64.b64encode(image_file.read()).decode('utf-8')
      count = 0

      tensor_list = results[0].boxes.xyxy

      sorted_tensor_list = sorted(tensor_list, key=lambda x: x[1])

      # Group the sorted_tensor_list based on the absolute difference between the second values of adjacent torch.tensor objects
      groups = []
      current_group = [sorted_tensor_list[0]]
      for i in range(1, len(sorted_tensor_list)):
          if abs(sorted_tensor_list[i][1] - sorted_tensor_list[i-1][1]) < 200:
              current_group.append(sorted_tensor_list[i])
          else:
              groups.append(current_group)
              current_group = [sorted_tensor_list[i]]
      groups.append(current_group)  # Add the last group

      # Sort each group based on the first value of each torch.tensor object in increasing order
      sorted_groups = [sorted(group, key=lambda x: x[0]) for group in groups]

      for i, group in enumerate(sorted_groups):
          for coord in group:
              xmin, ymin, xmax, ymax = map(int, coord.tolist())  # Convert tensor to list of ints
              # Crop the detected object from the original image
              cropped_object = original_image.crop((xmin, ymin, xmax, ymax))
              # Optionally, display the cropped object
              # display(cropped_object)
              # Save the cropped object with a path that includes the output directory
              cropped_object.save(os.path.join(output_dir, f'cropped_object_{count}.jpg'))
              count += 1

      ocr = OCREngine(modelType=ModelTypes.Easyocr)

      images_dir = '/content/kannada_output'

      if not os.path.exists(images_dir):
          print(f"Directory {images_dir} does not exist.")
      else:
          image_files = sorted(os.listdir(images_dir))  # Sorted to maintain order
          text = ""
          for image_file in image_files:
              image_path = os.path.join(images_dir, image_file)
              if not os.path.isfile(image_path):
                  continue
              image_text = ocr.get_text_from_file(image_path)
              text += f" {image_text}"
          print(text)
      tts = gTTS(text, lang='kn')

      tts.save("kannada_speech.mp3")

      print("Audio file saved as kannada_speech.mp3")
      output_with_model = text
      ocr = OCREngine(modelType=ModelTypes.Easyocr)
      text = ocr.get_text_from_file("/content/static/uploads/input.jpg")
      output_without_model = text

      return render_template('index.html', output_without_model=output_without_model, output_with_model=output_with_model, image_data=image_data, clipart_data=clipart_base64)
    else:
        # Just render the form page if the request is GET
        return render_template('index.html')

@app.route("/play", methods=['GET', 'POST'])
def play():
    # Define the path to the audio file you want to send
    audio_file_path = "/content/kannada_speech.mp3"  # Update this with the actual file path

    # Send the audio file in the response for playback
    return send_file(audio_file_path, as_attachment=True)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()



Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········
 * ngrok tunnel "https://a2c6-34-48-62-237.ngrok-free.app" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [37]:
import os

# Define the path to the 'static' directory
static_dir = 'static'

# Define the path to the 'uploads' directory within the 'static' directory
uploads_dir = os.path.join(static_dir, 'uploads')

# Create the 'uploads' directory if it doesn't exist
if not os.path.exists(uploads_dir):
    os.makedirs(uploads_dir)
    print(f"Directory '{uploads_dir}' created successfully.")
else:
    print(f"Directory '{uploads_dir}' already exists.")


Directory 'static/uploads' created successfully.


In [ ]:
import os
from ultralytics import YOLO

# Define the absolute path to the model file
model_path = os.path.abspath('/content/best.pt')

try:
    # Load the YOLOv5 model
    model = YOLO(model_path)
    print("YOLOv5 model loaded successfully.")
except Exception as e:
    print("Error loading YOLOv5 model:", e)
